In [1]:
import pandas as pd
df = pd.read_excel('RD.xlsx')
df.head()

,Listing_Title,City,Area,Location,Zone,Propery_Type,Size_In_Sqft,Carpet_Area_Sqft,Bedrooms,Bathrooms,...,Property_Age,Brokerage,Floor_No,Total_floors_In_Building,Maintenance_Charge,Nearby_Facilities,Recomened for,Muncipla Water Or Bore Water,Type of Society,Road Connectivity
0,1 BHK Flat for Rent,Nagpur,shatabdisquare,"Shatabdi Square, Nagpur, Maharashtra",south zone,1 BHK Flat,800 .sqft,NaN,1,1,...,Less than 5 years,No,2 Floor,2 Floors,No,"school,hospital",family,Borewell,Non-Gated,under 2 km
1,1 BHK Flat For Rent,Nagpur,manewada,"Manewada, Nagpur, Maharashtra",south zone,1 BHK Flat,700 .sqft,NaN,1,1,...,new Construction,No,2 Floor,3 Floors,No,"school,hospital",family,Both,Non-Gated,under 1 km
2,1 RK Flat for Rent,Nagpur,hudkeswar,Devashree Krishnam Nagari Location Hudkeshwar ...,south zone,1 RK,1000.sqft,NaN,1,1,...,NaN,No,NaN,NaN,No,"School,Hospital, metro station, mall",family,Borewell,Gated,under 2 km
3,2 BHK Flat for Rent,Nagpur,manish nagar,"Nandita Apartment manish nagar , New Manish Na...",south zone,2 BHK Flat,500 .sqft,NaN,2,2,...,NaN,No,NaN,NaN,No,"school, hospital, metro station, airport",NaN,Borewell,Non-Gated,under 2 km
4,1 BHK Flat for Rent,Nagpur,beltarodi,"SDPL Aashray Beltarodi II, Beltarodi, Nagpur",south zone,1 BHK Flat,525.sqft,NaN,1,1,...,NaN,No,NaN,NaN,No,"school, hospital, metro station, airport",NaN,Municipal,Gated,under 1 km


In [2]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
import seaborn as sns  
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import optuna
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')  # ignore notifications
import sys
import os


In [3]:
df['Frurnishing_Status'].unique()
mapping = {
    'Semi Furnished': 'Semi Furnished',
    'Semi-Furnished': 'Semi Furnished',
    'Furnished': 'Fully Furnished',
    'Unfurnished': 'Unfurnished',
    'Fully Furnished': 'Fully Furnished'
}
df['Frurnishing_Status'] = df['Frurnishing_Status'].map(mapping)
df['Frurnishing_Status'].value_counts()

Frurnishing_Status
Semi Furnished     390
Fully Furnished    320
Unfurnished        290
Name: count, dtype: int64

In [4]:
df['Recomened for'].unique()
mapping = {
    'family': 'Family',
    'Family': 'Family',
    'bachelor': 'Bachelors',
    'batchelor': 'Bachelors',
    'batchlor': 'Bachelors',
    'Bachelors': 'Bachelors',
    'family,company': 'Family and Company',
    'family,batchelor': 'Family and Bachelors',
    'family, batchelor': 'Family and Bachelors',
    'Anyone': 'Anyone',
    '-1': 'Anyone',
    'family,batchelor,company': 'Anyone',
}

df['Recomened for'] = df['Recomened for'].map(mapping)
df['Recomened for'].value_counts()

Recomened for
Anyone                  336
Family                  285
Bachelors               218
Family and Company       36
Family and Bachelors     30
Name: count, dtype: int64

In [5]:
df['Amenities'].unique()
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Step 1: Replace -1 and NaN with empty string
df['Amenities'] = df['Amenities'].replace(-1, '').fillna('')

# Step 2: Convert to lowercase, remove extra spaces, and correct typos
df['Amenities'] = df['Amenities'].astype(str).str.lower()
df['Amenities'] = df['Amenities'].str.replace('geted', 'gated')
df['Amenities'] = df['Amenities'].str.replace('cummunity', 'community')
df['Amenities'] = df['Amenities'].str.replace('backpu', 'backup')
df['Amenities'] = df['Amenities'].str.replace('allowd', 'allowed')
df['Amenities'] = df['Amenities'].str.replace('alloed', 'allowed')
df['Amenities'] = df['Amenities'].str.replace('paower', 'power')
df['Amenities'] = df['Amenities'].str.replace('intercome', 'intercom')
df['Amenities'] = df['Amenities'].str.replace('sporty', 'sports')
df['Amenities'] = df['Amenities'].str.replace('sysytem', 'system')
df['Amenities'] = df['Amenities'].str.replace(' ,', ',')  # remove space before commas

# Step 3: Remove extra spaces around commas and values
df['Amenities'] = df['Amenities'].str.replace(r'\s*,\s*', ',', regex=True)
df['Amenities'] = df['Amenities'].str.strip()

# Step 4: Convert comma-separated strings to list
df['Amenities'] = df['Amenities'].apply(lambda x: x.split(',') if x else [])

# Step 5: Apply MultiLabelBinarizer
mlb = MultiLabelBinarizer()
amenities_encoded = pd.DataFrame(mlb.fit_transform(df['Amenities']), columns=mlb.classes_)

# Step 6: Concatenate with original dataframe
df = pd.concat([df.drop(columns=['Amenities']), amenities_encoded], axis=1)

# Optional: Check the result
print(df.head())
# Check for binary columns (containing only 0, 1, or NaN)
binary_columns = [col for col in df.columns if df[col].dropna().isin([0, 1]).all()]

print("Binary columns are:")
print(binary_columns)
# Define similar columns you want to merge
merge_map = {
    'gas_pipeline': ['gas pipeline', 'gas pipline'],
    'sports_facility': ['sport facility', 'sports facility','basket ball court'],
    'kids_area': ['kids area', 'kids arear', 'kids pool'],
    'power_backup': ['power backup', 'powert backup'],
    'Garden': ['close to jain temple', 'garden'],
    'Fire Support': ['fire fighting', 'fire fighting system'],
    'Parking': ['parking', 'paeking']
}

# Merge each group
for new_col, cols_to_merge in merge_map.items():
    df[new_col] = df[cols_to_merge].max(axis=1)
    df.drop(columns=cols_to_merge, inplace=True)

# Print remaining binary columns
remaining_binary_cols = [col for col in df.columns if df[col].dropna().isin([0, 1]).all()]
print("Remaining binary columns:", remaining_binary_cols)

           Listing_Title    City            Area  \
0   1 BHK Flat for Rent   Nagpur  shatabdisquare   
1  1 BHK Flat  For Rent   Nagpur        manewada   
2     1 RK Flat for Rent  Nagpur       hudkeswar   
3    2 BHK Flat for Rent  Nagpur    manish nagar   
4    1 BHK Flat for Rent  Nagpur       beltarodi   

                                            Location        Zone Propery_Type  \
0               Shatabdi Square, Nagpur, Maharashtra  south zone   1 BHK Flat   
1                      Manewada, Nagpur, Maharashtra  south zone   1 BHK Flat   
2  Devashree Krishnam Nagari Location Hudkeshwar ...  south zone         1 RK   
3  Nandita Apartment manish nagar , New Manish Na...  south zone   2 BHK Flat   
4       SDPL Aashray Beltarodi II, Beltarodi, Nagpur  south zone   1 BHK Flat   

  Size_In_Sqft Carpet_Area_Sqft  Bedrooms  Bathrooms  ...  parking  \
0    800 .sqft              NaN         1          1  ...        0   
1    700 .sqft              NaN         1          1  ...   

In [6]:
df['Nearby_Facilities'].unique()
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Step 1: Replace -1 and NaN with empty strings
df['Nearby_Facilities'] = df['Nearby_Facilities'].replace(-1, '').fillna('')

# Step 2: Convert to lowercase and fix typos (as needed)
df['Nearby_Facilities'] = df['Nearby_Facilities'].astype(str).str.lower()
df['Nearby_Facilities'] = df['Nearby_Facilities'].str.replace('hospit[l|a]', 'hospital', regex=True)
df['Nearby_Facilities'] = df['Nearby_Facilities'].str.replace('schooll', 'school')
df['Nearby_Facilities'] = df['Nearby_Facilities'].str.replace(' ,', ',')  # remove space before commas

# Step 3: Remove extra spaces around commas and values
df['Nearby_Facilities'] = df['Nearby_Facilities'].str.replace(r'\s*,\s*', ',', regex=True)
df['Nearby_Facilities'] = df['Nearby_Facilities'].str.strip()

# Step 4: Convert comma-separated strings to list
df['Nearby_Facilities'] = df['Nearby_Facilities'].apply(lambda x: x.split(',') if x else [])

# Step 5: Apply MultiLabelBinarizer
mlb_nf = MultiLabelBinarizer()
nearby_encoded = pd.DataFrame(mlb_nf.fit_transform(df['Nearby_Facilities']), columns=mlb_nf.classes_)

# Step 6: Concatenate with original dataframe
df = pd.concat([df.drop(columns=['Nearby_Facilities']), nearby_encoded], axis=1)

# Optional: Check result
print(df.head())
# Print remaining binary columns
remaining_binary_cols = [col for col in df.columns if df[col].dropna().isin([0, 1]).all()]
print("Remaining binary columns:", remaining_binary_cols)
# Define similar columns you want to merge
merge_map = {
    'ATM_Near_me': ['atm'],
    'Airport_Near_me': ['airport'],
    'Bus_Stop__Near_me': ['bus stop'],
    'Hospital_Near_me': ['hospital','hospitall'],
    'Mall_Near_me': ['mall'],
    'Market_Near_me': ['market'],
    'Metro_Station_Near_me': ['metro station'],
    'Park_Near_me': ['park'],
    'School_Near_me': ['school'],
}

# Merge each group
for new_col, cols_to_merge in merge_map.items():
    df[new_col] = df[cols_to_merge].max(axis=1)
    df.drop(columns=cols_to_merge, inplace=True)

# Print remaining binary columns
remaining_binary_cols = [col for col in df.columns if df[col].dropna().isin([0, 1]).all()]
print("Remaining binary columns:", remaining_binary_cols)

           Listing_Title    City            Area  \
0   1 BHK Flat for Rent   Nagpur  shatabdisquare   
1  1 BHK Flat  For Rent   Nagpur        manewada   
2     1 RK Flat for Rent  Nagpur       hudkeswar   
3    2 BHK Flat for Rent  Nagpur    manish nagar   
4    1 BHK Flat for Rent  Nagpur       beltarodi   

                                            Location        Zone Propery_Type  \
0               Shatabdi Square, Nagpur, Maharashtra  south zone   1 BHK Flat   
1                      Manewada, Nagpur, Maharashtra  south zone   1 BHK Flat   
2  Devashree Krishnam Nagari Location Hudkeshwar ...  south zone         1 RK   
3  Nandita Apartment manish nagar , New Manish Na...  south zone   2 BHK Flat   
4       SDPL Aashray Beltarodi II, Beltarodi, Nagpur  south zone   1 BHK Flat   

  Size_In_Sqft Carpet_Area_Sqft  Bedrooms  Bathrooms  ...  Parking airport  \
0    800 .sqft              NaN         1          1  ...        0       0   
1    700 .sqft              NaN         1   

In [7]:
print(df['Carpet_Area_Sqft'])
df['Carpet_Area_Sqft'] = df['Carpet_Area_Sqft'].str.replace(r'\.?\s*sqft', '', regex=True).str.strip()
print(df['Carpet_Area_Sqft'])

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
995    1100 sqft
996     750 sqft
997     900 sqft
998    1100 sqft
999     750.sqft
Name: Carpet_Area_Sqft, Length: 1000, dtype: object
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
995    1100
996     750
997     900
998    1100
999     750
Name: Carpet_Area_Sqft, Length: 1000, dtype: object


In [8]:
import re
print(df['Total_floors_In_Building'])
df['Total_floors_In_Building'] = df['Total_floors_In_Building'].str.replace(r'\.?\s*floors?', '', regex=True, flags=re.IGNORECASE).str.strip()
print(df['Total_floors_In_Building'])

0      2 Floors
1      3 Floors
2           NaN
3           NaN
4           NaN
         ...   
995    9 Floors
996    5 Floors
997    3 Floors
998    7 Floors
999    5 Floors
Name: Total_floors_In_Building, Length: 1000, dtype: object
0        2
1        3
2      NaN
3      NaN
4      NaN
      ... 
995      9
996      5
997      3
998      7
999      5
Name: Total_floors_In_Building, Length: 1000, dtype: object


In [9]:

import re
print(df['Floor_No'])
df['Floor_No'] = df['Floor_No'].str.replace(r'\.?\s*floors?', '', regex=True, flags=re.IGNORECASE).str.strip()
df['Floor_No'] = df['Floor_No'].str.replace(r'Ground', 'G', regex=True, flags=re.IGNORECASE)
print(df['Floor_No'])

0      2 Floor
1      2 Floor
2          NaN
3          NaN
4          NaN
        ...   
995    1 Floor
996    2 Floor
997    2 Floor
998    2 Floor
999    3 Floor
Name: Floor_No, Length: 1000, dtype: object
0        2
1        2
2      NaN
3      NaN
4      NaN
      ... 
995      1
996      2
997      2
998      2
999      3
Name: Floor_No, Length: 1000, dtype: object


In [11]:
df['Listing_Title'].unique()
mapping = {
    '1 BHK Flat for Rent ':'1 BHK Flat for Rent',
    '1 BHK Flat  For Rent ':'1 BHK Flat for Rent',
    '1 BHK Flat for Rent':'1 BHK Flat for Rent',
    '1 BHK Flat\xa0for Rent':'1 BHK Flat for Rent',
    
    '3 BHK Flat for rent':'3 BHK Flat for rent',
    '3 BHK Apartment for Rent':'3 BHK Flat for rent',
    '3 BHK Flat\xa0for Rent':'3 BHK Flat for rent',
    
    '2 BHK Flat for Rent':'2 BHK Flat for Rent',
    '2 BHK  Flat for Rent':'2 BHK Flat for Rent',
    '2 BHK Flat for Rent ':'2 BHK Flat for Rent',
    '2 BHK Flat\xa0for Rent':'2 BHK Flat for Rent',

    '4 BHK Flat for rent':'4 BHK Flat for rent',

    '1 RK Flat for Rent':'1 RK Flat for Rent',
    'Studio Apartment for Rent':'Studio Apartment for Rent',
    
    '1 BHK Independent House\xa0for Rent':'1 BHK Independent House for Rent',
    '1 BHK Independent House for Rent':'1 BHK Independent House for Rent',
    
    '3 BHK Independent House\xa0for Rent':'3 BHK Independent House for Rent',
    
    '4 BHK Independent House\xa0for Rent':'4 BHK Independent House for Rent',
    
    '5 BHK Independent House\xa0for Rent':'5 BHK Independent House for Rent',

    '1 BHK Independent Builder Floor\xa0for Rent':'1 BHK Independent Builder Floor for Rent',
    
    '2 BHK Independent Builder Floor for Rent':'2 BHK Independent Builder Floor for Rent',
    '2 BHK Independent Builder Floor\xa0for Rent':'2 BHK Independent Builder Floor for Rent',
    
    '3 BHK Independent Builder Floor for Rent':'3 BHK Independent Builder Floor for Rent',
    
    '4 BHK Independent Builder Floor\xa0for Rent':'4 BHK Independent Builder Floor for Rent',

    '1 BHK Villa\xa0for Rent':'1 BHK Villa for Rent',
    
    '3 BHK Villa\xa0for Rent':'3 BHK Villa for rent',
    '3 BHK villa for rent':'3 BHK Villa for rent',
    
    '4 BHK Villa\xa0for Rent':'4 BHK Villa for Rent',

    '1 RK Independent House\xa0for Rent':'1 RK Independent House for Rent',
    '1 RK Independent house\xa0for Rent':'1 RK Independent House for Rent',

    '3 BHK Duplex\xa0for Rent':'3 BHK Duplex for Rent',

    
}

df['Listing_Title'] = df['Listing_Title'].map(mapping)
df['Listing_Title'].value_counts()

Listing_Title
2 BHK Flat for Rent                         231
Studio Apartment for Rent                   218
3 BHK Flat for rent                         208
1 RK Flat for Rent                          183
3 BHK Independent House for Rent             31
1 BHK Flat for Rent                          14
1 BHK Independent House for Rent             12
1 RK Independent House for Rent              12
2 BHK Independent Builder Floor for Rent     10
1 BHK Independent Builder Floor for Rent      7
3 BHK Villa for rent                          6
3 BHK Independent Builder Floor for Rent      3
4 BHK Independent Builder Floor for Rent      2
4 BHK Villa for Rent                          2
1 BHK Villa for Rent                          2
3 BHK Duplex for Rent                         2
5 BHK Independent House for Rent              2
4 BHK Independent House for Rent              2
4 BHK Flat for rent                           1
Name: count, dtype: int64

In [12]:
df['Zone'].unique()
mapping = {
    'south zone':'South Zone',
    'east zone':'East Zone',
    'west zone':'West Zone',
    'West':'West Zone',
    'north zone':'North Zone',
    'rural':'rural',
}
df['Zone'] = df['Zone'].map(mapping)
df['Zone'].value_counts()

Zone
South Zone    318
West Zone     257
East Zone     219
North Zone    204
rural           2
Name: count, dtype: int64

In [13]:
# Extract Room (BHK, RK, Studio)
df['Room'] = df['Listing_Title'].str.extract(r'(\d+\s*BHK|\d+\s*RK|Studio Apartment)', expand=False)

# Extract Type (Flat, Independent House, etc.)
df['Type'] = df['Listing_Title'].str.extract(
    r'(Flat|Independent House|Independent Builder Floor|Villa|Duplex|Studio Apartment)', 
    expand=False
)

df.drop(columns=['Listing_Title'], inplace=True)
df.drop(columns=['Propery_Type'], inplace=True)

In [25]:
df['Property_Age'].unique()


age_mapping = {
    'Less than 5 years': 5,
    'More than 20 years': 20,
    'New Property': 0,
    'new Construction': 0,
    ' ': np.nan  
}
df['Property_Age'] = df['Property_Age'].replace(age_mapping)
df['Property_Age'] = df['Property_Age'].apply(
    lambda x: int(re.sub(r'\D', '', str(x))) if re.search(r'\d', str(x)) else x
)
df['Property_Age'] = pd.to_numeric(df['Property_Age'], errors='coerce').fillna(0).astype(int)
df['Property_Age'].value_counts()

Property_Age
0     176
5     117
1      88
3      67
10     60
2      52
15     52
4      49
11     48
9      46
12     45
6      44
8      41
14     40
13     38
7      37
Name: count, dtype: int64

In [27]:
import re

def convert_to_km(value):
    if pd.isna(value):
        return None
    
    value = str(value).lower().strip()
    
    # Extract number
    num_match = re.search(r'(\d+)', value)
    if not num_match:
        return None
    num = float(num_match.group(1))
    
    # Check unit
    if 'm' in value and 'km' not in value:  # meters
        return num / 1000
    else:  # kilometers
        return num

df['Road Connectivity'] = df['Road Connectivity'].apply(convert_to_km)
df['Road Connectivity'].value_counts()

Road Connectivity
1.0    467
2.0    261
5.0    198
0.5     32
4.0     25
3.0     15
6.0      2
Name: count, dtype: int64

In [29]:
df['Brokerage'].unique()
mapping = {
    'No':'NO',
    'yes':'YES',
    'Yes':'YES'
}
df['Brokerage'] = df['Brokerage'].map(mapping)
df['Brokerage'].value_counts()

Brokerage
NO     546
YES    454
Name: count, dtype: int64

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   City                          1000 non-null   object 
 1   Area                          1000 non-null   object 
 2   Location                      1000 non-null   object 
 3   Zone                          1000 non-null   object 
 4   Size_In_Sqft                  1000 non-null   object 
 5   Carpet_Area_Sqft              772 non-null    object 
 6   Bedrooms                      1000 non-null   int64  
 7   Bathrooms                     1000 non-null   int64  
 8   Balcony                       852 non-null    float64
 9   Frurnishing_Status            1000 non-null   object 
 10  Number_Of_Amenities           1000 non-null   int64  
 11  Rent_Price                    1000 non-null   float64
 12  Property_Age                  1000 non-null   int32  
 13  Brok

In [33]:
df = df.drop('Location', axis=1)

In [35]:
df = df.replace(" ", 0).fillna(0)

In [37]:
null_counts = df.isnull().sum()
print(null_counts)

City                            0
Area                            0
Zone                            0
Size_In_Sqft                    0
Carpet_Area_Sqft                0
Bedrooms                        0
Bathrooms                       0
Balcony                         0
Frurnishing_Status              0
Number_Of_Amenities             0
Rent_Price                      0
Property_Age                    0
Brokerage                       0
Floor_No                        0
Total_floors_In_Building        0
Maintenance_Charge              0
Recomened for                   0
Muncipla Water Or Bore Water    0
Type of Society                 0
Road Connectivity               0
gated community                 0
gym                             0
intercom                        0
lift                            0
pet allowed                     0
pool                            0
security                        0
water supply                    0
wifi                            0
gas_pipeline  

In [39]:
print("--- Feature Engineering: One-Hot Encoding Categorical Features ---")
categorical_features = ['City','Area', 'Zone', 'Frurnishing_Status', 'Brokerage', 'Maintenance_Charge', 'Recomened for', 'Muncipla Water Or Bore Water', 'Type of Society', 'Room', 'Type']
for feature in categorical_features:
    if feature in df.columns:
        df = pd.get_dummies(df, columns=[feature], drop_first=True)
        print(f"One-hot encoded '{feature}'.")
    else:
        print(f"Warning: Categorical feature '{feature}' not found in DataFrame for encoding.")


# Transform 'Rent' using natural logarithm to reduce skewness
print("--- Feature Engineering: Log Transformation of 'Rent' ---")
# Add a small constant to Rent to avoid log(0) if any rent value is 0
df['Rent_Price'] = np.log1p(df['Rent_Price']) # Using log1p which is log(1+x)
print("Applied log transformation to 'Rent' column.")

print("\n--- DataFrame after Feature Engineering ---")
print(df.head())
print(df.info())

--- Feature Engineering: One-Hot Encoding Categorical Features ---
One-hot encoded 'City'.
One-hot encoded 'Area'.
One-hot encoded 'Zone'.
One-hot encoded 'Frurnishing_Status'.
One-hot encoded 'Brokerage'.
One-hot encoded 'Maintenance_Charge'.
One-hot encoded 'Recomened for'.
One-hot encoded 'Muncipla Water Or Bore Water'.
One-hot encoded 'Type of Society'.
One-hot encoded 'Room'.
One-hot encoded 'Type'.
--- Feature Engineering: Log Transformation of 'Rent' ---
Applied log transformation to 'Rent' column.

--- DataFrame after Feature Engineering ---
  Size_In_Sqft Carpet_Area_Sqft  Bedrooms  Bathrooms  Balcony  \
0    800 .sqft                0         1          1      0.0   
1    700 .sqft                0         1          1      0.0   
2    1000.sqft                0         1          1      0.0   
3    500 .sqft                0         2          2      0.0   
4     525.sqft                0         1          1      0.0   

   Number_Of_Amenities  Rent_Price  Property_Age Floo

In [42]:
# Define features (X) and target (y)
X = df.drop('Rent_Price', axis=1)
y = df['Rent_Price']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"\nData split into training (80%) and testing (20%) sets.")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


Data split into training (80%) and testing (20%) sets.
X_train shape: (800, 123), y_train shape: (800,)
X_test shape: (200, 123), y_test shape: (200,)


In [44]:
numerical_cols = ['Size_In_Sqft', 'Carpet_Area_Sqft', 'Bedrooms', 'Bathrooms', 'Balcony',
'Number_Of_Amenities', 'Floor_No', 'Total_floors_In_Building',
'Road_Connectivity', 'gated_community', 'gym', 'intercom', 'lift', 'pet_allowed', 'pool',
'security', 'water_supply', 'wifi', 'gas_pipeline', 'sports_facility', 'kids_area',
'power_backup', 'Garden', 'Fire_Support', 'Parking', 'ATM_Near_me', 'Airport_Near_me',
'Bus_Stop__Near_me', 'Hospital_Near_me', 'Mall_Near_me', 'Market_Near_me',
'Metro_Station_Near_me', 'Park_Near_me', 'School_Near_me', 'Property_Age']
# Filter numerical_cols to only include columns actually present in X_train
numerical_cols_present = [col for col in numerical_cols if col in X_train.columns]

# 1. Replace empty strings with NaN
X_train = X_train.replace(r'^\s*$', np.nan, regex=True)
X_test = X_test.replace(r'^\s*$', np.nan, regex=True)

# 2. Convert all numeric columns to float
X_train[numerical_cols_present] = X_train[numerical_cols_present].apply(pd.to_numeric, errors='coerce')
X_test[numerical_cols_present] = X_test[numerical_cols_present].apply(pd.to_numeric, errors='coerce')

# 3. Fill NaN with median for each column
for col in numerical_cols_present:
    median_val = X_train[col].median()
    X_train[col].fillna(median_val, inplace=True)
    X_test[col].fillna(median_val, inplace=True)

# 4. Scale the numeric columns
scaler = StandardScaler()
X_train[numerical_cols_present] = scaler.fit_transform(X_train[numerical_cols_present])
X_test[numerical_cols_present] = scaler.transform(X_test[numerical_cols_present])

In [83]:
from sklearn.metrics import r2_score, mean_squared_error
print("\n--- Model Training: Random Forest Regressor ---")
# Initialize and train the Random Forest Regressor model
# Using parameters that generally give good results, but further tuning can be done.
rf_model = RandomForestRegressor(n_estimators=450,n_jobs=-1,random_state=42)
rf_model.fit(X_train, y_train)
print("Random Forest Regressor model trained.")

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"\n--- Model Evaluation (Random Forest) ---")
print(f"R-squared (R^2) Score: {r2_rf:.4f}")
print(f"Mean Squared Error (MSE): {mse_rf:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_rf:.4f}")


--- Model Training: Random Forest Regressor ---
Random Forest Regressor model trained.

--- Model Evaluation (Random Forest) ---
R-squared (R^2) Score: 0.5131
Mean Squared Error (MSE): 0.1036
Root Mean Squared Error (RMSE): 0.3219


In [84]:
import pandas as pd
import numpy as np

# Get feature importances
importances = rf_model.feature_importances_

# If X_train is a DataFrame
feature_names = X_train.columns




# Convert to percentage
importances_percentage = 100 * importances / importances.sum()

# Create DataFrame of feature importances
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance (%)': importances_percentage
}).sort_values(by='Importance (%)', ascending=False)

print(feature_importance_df)


                      Feature  Importance (%)
1            Carpet_Area_Sqft       13.246421
3                   Bathrooms        5.930747
4                     Balcony        5.627198
6                Property_Age        5.130740
8    Total_floors_In_Building        4.433494
..                        ...             ...
41    Area_chandrakiran nagar        0.001450
117               Type_Duplex        0.000497
72               Area_narsala        0.000124
74         Area_parvati nagar        0.000000
0                Size_In_Sqft        0.000000

[123 rows x 2 columns]


In [61]:
feature_importance_df.to_excel("predicted.xlsx", index=False)

In [87]:
# --- Saving the trained model and scaler ---
import joblib
print("\n--- Saving Model and Scaler ---")
try:
    joblib.dump(rf_model, 'm3.pkl')
    joblib.dump(scaler, 's3.pkl')
    joblib.dump(X.columns.tolist(), 'f3.pkl') # Save feature names for consistency
    print("Model, scaler, and feature names saved successfully.")
except Exception as e:
    print(f"Error saving model/scaler: {e}")


--- Saving Model and Scaler ---
Model, scaler, and feature names saved successfully.


In [81]:
import joblib

# Save with compression
joblib.dump(rf_model, "m.pkl", compress=3)


['m.pkl']